In [1]:
import os

import pandas as pd
import numpy as np

import hopsworks

from datetime import datetime, timedelta
from pytz import timezone

from src.webscraping_a import (
    scrape_to_dataframe,
    convert_columns,
    combine_home_visitor,  
    get_todays_matchups,
)

from src.data_processing import (
    process_games,
    add_TARGET,
)

from src.feature_engineering import (
    process_features,
)

from src.hopsworks_utils import (
    save_feature_names,
    convert_feature_names,
)

import json

from pathlib import Path  #for Windows/Linux compatibility
DATAPATH = Path(r'data')

In [2]:
from dotenv import load_dotenv

load_dotenv()

try:
    HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']
except:
    raise Exception('Set environment variable HOPSWORKS_API_KEY')

In [3]:
project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY)
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/3350
Connected. Call `.close()` to terminate connection gracefully.


In [4]:
rolling_stats_fg = fs.get_feature_group(
    name="rolling_stats",
    version=1,
)

In [5]:
BASE_FEATURES = ['game_date_est',
 'game_id',
 'home_team_id',
 'visitor_team_id',
 'season',
 'pts_home',
 'fg_pct_home',
 'ft_pct_home',
 'fg3_pct_home',
 'ast_home',
 'reb_home',
 'pts_away',
 'fg_pct_away',
 'ft_pct_away',
 'fg3_pct_away',
 'ast_away',
 'reb_away',
 'home_team_wins',
]

ds_query = rolling_stats_fg.select(BASE_FEATURES)
df_old = ds_query.read()
df_old = convert_feature_names(df_old)
df_old[df_old['PTS_home'] == 0]


2023-01-31 08:30:10,072 INFO: USE `nba_predictor_featurestore`
2023-01-31 08:30:10,441 INFO: SELECT `fg0`.`game_date_est` `game_date_est`, `fg0`.`game_id` `game_id`, `fg0`.`home_team_id` `home_team_id`, `fg0`.`visitor_team_id` `visitor_team_id`, `fg0`.`season` `season`, `fg0`.`pts_home` `pts_home`, `fg0`.`fg_pct_home` `fg_pct_home`, `fg0`.`ft_pct_home` `ft_pct_home`, `fg0`.`fg3_pct_home` `fg3_pct_home`, `fg0`.`ast_home` `ast_home`, `fg0`.`reb_home` `reb_home`, `fg0`.`pts_away` `pts_away`, `fg0`.`fg_pct_away` `fg_pct_away`, `fg0`.`ft_pct_away` `ft_pct_away`, `fg0`.`fg3_pct_away` `fg3_pct_away`, `fg0`.`ast_away` `ast_away`, `fg0`.`reb_away` `reb_away`, `fg0`.`home_team_wins` `home_team_wins`
FROM `nba_predictor_featurestore`.`rolling_stats_1` `fg0`


,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
1384,2023-01-31,22200765,1610612752,1610612747,2022,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,0
3042,2023-01-31,22200767,1610612749,1610612766,2022,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,0
8239,2023-01-31,22200764,1610612739,1610612748,2022,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,0
8907,2023-01-31,22200766,1610612741,1610612746,2022,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,0
21628,2023-01-31,22200768,1610612743,1610612740,2022,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,0


In [6]:
df_old[df_old['GAME_ID'] > 22200751]

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
1384,2023-01-31,22200765,1610612752,1610612747,2022,0,0.00000,0.0000,0.000000,0,0,0,0.00000,0.0000,0.000000,0,0,0
1705,2023-01-30,22200759,1610612744,1610612760,2022,128,51.09375,80.0000,42.593750,37,44,120,49.50000,85.0000,45.812500,23,43,1
1755,2023-01-29,22200754,1610612746,1610612739,2022,99,41.59375,85.1875,10.500000,17,45,122,56.40625,73.6875,60.593750,35,35,0
3042,2023-01-31,22200767,1610612749,1610612766,2022,0,0.00000,0.0000,0.000000,0,0,0,0.00000,0.0000,0.000000,0,0,0
3825,2023-01-30,22200756,1610612753,1610612755,2022,119,42.40625,80.0000,37.906250,23,50,109,47.09375,78.3125,36.687500,28,45,1
5048,2023-01-30,22200763,1610612737,1610612757,2022,125,46.81250,80.0000,43.312500,26,45,129,54.40625,88.8750,47.500000,24,32,0
5716,2023-01-29,22200753,1610612754,1610612763,2022,100,45.59375,90.5000,28.093750,21,38,112,48.31250,77.3125,24.296875,29,41,0
8239,2023-01-31,22200764,1610612739,1610612748,2022,0,0.00000,0.0000,0.000000,0,0,0,0.00000,0.0000,0.000000,0,0,0
8381,2023-01-29,22200752,1610612748,1610612766,2022,117,48.40625,73.8750,32.312500,26,36,122,54.18750,77.3125,37.500000,25,47,0
8727,2023-01-30,22200762,1610612761,1610612756,2022,106,44.90625,89.5000,27.296875,19,44,114,49.40625,81.0000,39.312500,28,42,0
